In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
def plan_install(df, shengcode, kw_year, year_col):
    #以一个省为例
    df = df.loc[df['Shengcode']==shengcode]
    #以2025年为例
    kw_year = kw_year * 10000
    df_sorted = df.sort_values(by = ['s2022','Score','KW2'], ascending=False)

    #剩下的需要分配的会动态更新，没分配一点就更新一点，初始的时候可供分配的还要考虑2022年已经布置的
    kwyear_updated = kw_year - df_sorted['KW2022'].sum()
    print(kwyear_updated)

    #分配之前需要先把2022年已经存在的先加进去
    df_sorted[year_col] = df_sorted['KW2022']

    for index, cell in df_sorted.iterrows():
        if(kwyear_updated > 0):    #如果待分配的有剩余，则开始分配
            #看这个格子的空间有多少，
            cell_room = max(cell['KW2'] - cell[year_col], 0)    #一个格子的最大容量还需要考虑格子里面已经有的, 如果这个格子里已经布置的容量已经超过预估的（kw2），则不再分配了
            #如果剩余够满足一个格子的空间则填满一个格子
            if(kwyear_updated > cell_room):
                cell[year_col] = cell[year_col] + cell_room 
                
            # 分配完一个格子，更新待分配的剩余量（kw2025_updated）
                kwyear_updated = kwyear_updated - cell_room
            # 如果不满足一个格子最大容量（kw2），则把剩余的一点全填到格子里
            else:
                cell[year_col] = cell[year_col] + kwyear_updated
            # 分配完一个格子，更新待分配的剩余量（kw2025_updated
                kwyear_updated = kwyear_updated - kwyear_updated 
            df_sorted.at[index, year_col] = cell[year_col]
            print(f'格子号：{index}, 还剩余待分配的：{kwyear_updated}，这个格子的容量{cell_room}')
        else:
            break
    return df_sorted

In [ ]:
# 打开风网格文件
path = r'D:\GoldWindProject2023\RENEWABLE_RESOURCE\Processing\MapsData\WIND_GRID10KM_20230914.shp'
points = gpd.read_file(path)

# 删除不需要的columns
# points.drop(columns=['hours','KWH2_2022','KW2_Unif_K'], inplace=True)  
points.drop(columns=['kw2025','kw2030','kw2035'], inplace=True) 

# 装机量分配

# 排除所有的2022年装机网格，并将小于0的数据设置为0
points['exclude_2022'] =  points['KW2'] - points['KW2022']
points.loc[points['exclude_2022']<0,['exclude_2022']] = 0

# 打开规划数据表格
path = r"D:\GoldWindProject2023\RENEWABLE_RESOURCE\00 原始数据\中国海上风光装机容量_20230319.xlsx"
plan_df = pd.read_excel(path, sheet_name="风电分省装机容量")
#提取shengcode
shengcode_list = plan_df['Shengcode'].drop_duplicates().dropna().tolist()

# 设置2022年筛选条件
points.loc[(points['KW2022']>0) & (points['exclude_2022']>0),['s2022']] = 1
points['s2022'].fillna(0,inplace=True)

points_copy = points.copy()
plan_df_col = ['2025年_低发展情景', '2030年_低发展情景', '2035年_低发展情景', '2040年_低发展情景', '2050年_低发展情景', '2060年_低发展情景']
col_name = []
for plan in plan_df_col:
    df_list = []
    for i in shengcode_list:
        kw_year = plan_df.loc[plan_df['Shengcode'] == i, plan].values[0]
        year_col = 'KW' + plan.split("年")[0]
        col_name.append(year_col)
        df = points_copy.loc[points_copy['Shengcode'] == i]
        df_list.append(plan_install(df, i, kw_year, year_col))
    df_kw = pd.concat(df_list)
    points_copy = points_copy.merge(df_kw.loc[:,['NID10a',year_col]], how = 'left', on ='NID10a')
points_copy.to_file(r'D:\GoldWindProject2023\RENEWABLE_RESOURCE\Processing\MapsData\wind_low_20240226.shp')

In [ ]:
# 打开风网格文件
path = r'D:\GoldWindProject2023\RENEWABLE_RESOURCE\Processing\MapsData\WIND_GRID10KM_20230914.shp'
points = gpd.read_file(path)

# 删除不需要的columns
points.drop(columns=['kw2025','kw2030','kw2035'], inplace=True) 

# 装机量分配

# 排除所有的2022年装机网格，并将小于0的数据设置为0
points['exclude_2022'] =  points['KW2'] - points['KW2022']
points.loc[points['exclude_2022']<0,['exclude_2022']] = 0

# 打开规划数据表格
path = r"D:\GoldWindProject2023\RENEWABLE_RESOURCE\00 原始数据\中国海上风光装机容量_20230319.xlsx"
plan_df = pd.read_excel(path, sheet_name="风电分省装机容量")
#提取shengcode
shengcode_list = plan_df['Shengcode'].drop_duplicates().dropna().tolist()

# 设置2022年筛选条件
points.loc[(points['KW2022']>0) & (points['exclude_2022']>0),['s2022']] = 1
points['s2022'].fillna(0,inplace=True)
points_copy = points.copy()
plan_df_col = ['2025年_中', '2030年_中', '2035年_中', '2040年_中', '2050年_中', '2060年_中']
col_name = []
for plan in plan_df_col:
    df_list = []
    for i in shengcode_list:
        kw_year = plan_df.loc[plan_df['Shengcode'] == i, plan].values[0]
        year_col = 'KW' + plan.split("年")[0]
        col_name.append(year_col)
        df = points_copy.loc[points_copy['Shengcode'] == i]
        df_list.append(plan_install(df, i, kw_year, year_col))
    df_kw = pd.concat(df_list)
    points_copy = points_copy.merge(df_kw.loc[:,['NID10a',year_col]], how = 'left', on ='NID10a')
points_copy.to_file(r'D:\GoldWindProject2023\RENEWABLE_RESOURCE\Processing\MapsData\wind_median_20240226.shp')

In [ ]:
# 打开风网格文件
path = r'D:\GoldWindProject2023\RENEWABLE_RESOURCE\Processing\MapsData\WIND_GRID10KM_20230914.shp'
points = gpd.read_file(path)

# 删除不需要的columns
points.drop(columns=['kw2025','kw2030','kw2035'], inplace=True) 

# 装机量分配

# 排除所有的2022年装机网格，并将小于0的数据设置为0
points['exclude_2022'] =  points['KW2'] - points['KW2022']
points.loc[points['exclude_2022']<0,['exclude_2022']] = 0

# 打开规划数据表格
path = r"D:\GoldWindProject2023\RENEWABLE_RESOURCE\00 原始数据\中国海上风光装机容量_20230319.xlsx"
plan_df = pd.read_excel(path, sheet_name="风电分省装机容量")
#提取shengcode
shengcode_list = plan_df['Shengcode'].drop_duplicates().dropna().tolist()

# 设置2022年筛选条件
points.loc[(points['KW2022']>0) & (points['exclude_2022']>0),['s2022']] = 1
points['s2022'].fillna(0,inplace=True)

points_copy = points.copy()
plan_df_col = ['2025年_高发展情景', '2030年_高发展情景', '2035年_高发展情景','2040年_高发展情景', '2050年_高发展情景', '2060年_高发展情景']
for plan in plan_df_col:
    df_list = []
    for i in shengcode_list:
        kw_year = plan_df.loc[plan_df['Shengcode'] == i, plan].values[0]
        year_col = 'KW' + plan.split("年")[0]
        df = points_copy.loc[points_copy['Shengcode'] == i]
        df_list.append(plan_install(df, i, kw_year, year_col))
    df_kw = pd.concat(df_list)
    points_copy = points_copy.merge(df_kw.loc[:,['NID10a',year_col]], how = 'left', on ='NID10a')
points_copy.to_file(r'D:\GoldWindProject2023\RENEWABLE_RESOURCE\Processing\MapsData\wind_high_20240226.shp')

In [ ]:
# # 数据检查
# # 按照省级代码检查
# path = r'D:\GoldWindProject2023\RENEWABLE_RESOURCE\Processing\MapsData\wind_low_20240226.shp'
# df = gpd.read_file(path)
# groupdf = df.loc[:,['Shengcode','KW2','KW2022','KW2025', 'KW2030', 'KW2035','KW2040', 'KW2050', 'KW2060']].groupby("Shengcode").agg('sum').reset_index()
# path = r"D:\GoldWindProject2023\RENEWABLE_RESOURCE\00 原始数据\中国海上风光装机容量_20230319.xlsx"
# plan_df = pd.read_excel(path, sheet_name="风电分省装机容量")
# groupdf = groupdf.merge(plan_df.loc[:, ['地区','Shengcode']], on='Shengcode', how='left')
# groupdf.to_excel(r"D:\GoldWindProject2023\RENEWABLE_RESOURCE\Processing\TableData\groupdf_low.xlsx")

In [ ]:
# 数据检查
# 按照省级代码检查
path = r'D:\GoldWindProject2023\RENEWABLE_RESOURCE\Processing\MapsData\wind_median_20240226.shp'
df = gpd.read_file(path)
groupdf = df.loc[:,['Shengcode','KW2','KW2022','KW2025', 'KW2030', 'KW2035','KW2040', 'KW2050', 'KW2060']].groupby("Shengcode").agg('sum').reset_index()
path = r"D:\GoldWindProject2023\RENEWABLE_RESOURCE\00 原始数据\中国海上风光装机容量_20230319.xlsx"
plan_df = pd.read_excel(path, sheet_name="风电分省装机容量")
groupdf = groupdf.merge(plan_df.loc[:, ['地区','Shengcode']], on='Shengcode', how='left')
groupdf.to_excel(r"D:\GoldWindProject2023\RENEWABLE_RESOURCE\Processing\TableData\groupdf_median.xlsx")

In [ ]:
# 数据检查
# 按照省级代码检查
path = r'D:\GoldWindProject2023\RENEWABLE_RESOURCE\Processing\MapsData\wind_high_20240226.shp'
df = gpd.read_file(path)
groupdf = df.loc[:,['Shengcode','KW2','KW2022','KW2025', 'KW2030', 'KW2035','KW2040', 'KW2050', 'KW2060']].groupby("Shengcode").agg('sum').reset_index()
path = r"D:\GoldWindProject2023\RENEWABLE_RESOURCE\00 原始数据\中国海上风光装机容量_20230319.xlsx"
plan_df = pd.read_excel(path, sheet_name="风电分省装机容量")
groupdf = groupdf.merge(plan_df.loc[:, ['地区','Shengcode']], on='Shengcode', how='left')
groupdf.to_excel(r"D:\GoldWindProject2023\RENEWABLE_RESOURCE\Processing\TableData\groupdf_high.xlsx")